In [1]:
import pandas as pd
import numpy as np


In [2]:
#Leemos los csv con los que trabajaremos
netflix_pd = pd.read_csv(r'Dataset\netflix_titles.csv')
hulu_pd = pd.read_csv(r'Dataset\hulu_titles.csv')
disney_pd = pd.read_csv(r'Dataset\disney_plus_titles.csv')
amazon_pd = pd.read_csv(r'Dataset\amazon_prime_titles.csv')
rating = pd.read_csv('rating-final.csv')


In [3]:
#Nos damos una idea de como estan compuesto los csv

netflix_pd.columns
netflix_pd.dtypes
netflix_pd.index
netflix_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [4]:
hulu_pd.columns
hulu_pd.dtypes
hulu_pd.index
hulu_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [5]:
disney_pd.columns
disney_pd.dtypes
disney_pd.index
disney_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [6]:
amazon_pd.columns
amazon_pd.dtypes
amazon_pd.index
amazon_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


Transformación de los datos
Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

De haber fechas, deberán tener el formato AAAA-mm-dd

Los campos de texto deberán estar en minúsculas, sin excepciones

El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [7]:
#Generamos la nueva columna 'id'
netflix_pd['id'] = 'n' + netflix_pd['show_id']
hulu_pd['id'] = 'h' + hulu_pd['show_id']
disney_pd['id'] = 'd' + disney_pd['show_id']
amazon_pd['id'] = 'a' + amazon_pd['show_id']

In [8]:
#combinamos los df habiendo verificando q los tipos de datos coinciden
df_concat = pd.concat([netflix_pd,disney_pd,amazon_pd,hulu_pd],axis=0)

In [9]:
#Transformacioens

#Los valores nulos del campo rating los remplazamos con “G”
df_concat['rating'].fillna('G',inplace=True)


#De haber fechas, deberán tener el formato AAAA-mm-dd
df_concat['date_added'] = pd.to_datetime(df_concat['date_added'],infer_datetime_format=True)

#Los campos de txt los pasamos para que estén todos en minúsculas:
columnstr= ['show_id', 'type', 'title', 'director', 'cast', 'country','rating', 'duration', 'listed_in', 'description', 'id']
        
for i in columnstr:
    df_concat[i] =  df_concat[i].str.lower()




In [10]:
#Divido la columna "duration" en duration_int y duration_type. 
duration = df_concat['duration'].str.split(expand=True)
duration.columns = ['duration_int','duration_type']

#concatenamos nuestras nuevas columnas a nuestro df original
df_concat = pd.concat([df_concat,duration],axis=1)

In [11]:
#traemos la columna score del csv de raiting
df_concat= df_concat.merge(rating, left_on='id',right_on='movieId')

In [12]:
#Eliminamos la columna sobrante movieId
df_concat.drop('movieId',axis=1,inplace=True)

In [13]:

#cambiamos el tipo de valor de la columna duration_int a integer
df_concat['duration_int'] = df_concat['duration_int'].astype('Int32',errors='ignore')


In [14]:
#Exportamos el csv
#df_concat.to_csv('Peliculas-F.csv',index=False)
